<a href="https://colab.research.google.com/github/francji1/01PR/blob/main/R/01PR_Ex07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01PR: Úvod do R - Cvičení 07

V tomto cvičení použijeme jeden z jenznámějších datasetů BostonHousing (možno získat z `library(mlbench)`  a `data(BostonHousing2)`).

Ukoly:

* Načtěte si data a vytvořte funkci, která pro libovolné dvě zadané numerické proměnné zobrazí jejich scatterplot, a velikost bodu bude odpovídat třetí numerické proměnné. Jako výchozí vezměte `cmdev`. Volitelně bude moci se přidat faktorová proměnná, která ovlivní barvu jednolivých bodů.

* Upravte předchozí funkci tak, aby umožnila vykreslit ve scatterplotu i linenární odhad (`y~x`)spolu s predikčním konfidenčním intervalem. Pro osu *x* použijte proměnnou `rm`, pro osu y `cmdev`, za faktorovou proměnnou (jak pro typ tak pro barvu bodu) zvolte `chas` a velikost bodu ať odpovídá `crim`.

* Napište cyklus, který vykreslí předchozí obrázek přes numerické proměnné na ose `x`: (`rm`,`lstat`,`crim`,`nox`) a na ose `y`: (`cmdev`) a pro vybranou faktorovou proměnnou vykreslí předchozí obrázky a uloží je do souboru.

* Napište funkci, která vybranou proměnnou vykeslí do mapy podle souřadnic (`lon` a `lat`) a na vstupu bude numerická proměnná podle jejíž hodnoty  se bude vykreslovat intenzita barvy.

In [ ]:
install.packages("mlbench")
install.packages("GGally")
library(mlbench)
library(tidyverse)
library(GGally)

In [ ]:
data(BostonHousing2)

In [ ]:
# ? BostonHousing2

**Variables** 

* crim:     per capita crime rate by town     
* zn: proportion of residential land zoned for lots over 25,000 sq.ft
* indus:    proportion of non-retail business acres per town 
* chas:     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) 
* nox:      nitric oxides concentration (parts per 10 million)     
* rm: average number of rooms per dwelling      
* age:      proportion of owner-occupied units built prior to 1940 
* dis:      weighted distances to five Boston employment centres   
* rad:      index of accessibility to radial highways 
* tax:      full-value property-tax rate per USD 10,000     
* ptratio:  pupil-teacher ratio by town
* b:  1000(B - 0.63)^2 where B is the proportion of blacks by town    
* lstat:    percentage of lower status of the population 
* medv:     median value of owner-occupied homes in USD 1000's   
* cmedv:  corrected median value of owner-occupied homes in USD 1000's 
* town:   name of town
* tract:  census tract
* lon:    longitude of census tract
* lat:    latitude of census tract 

In [ ]:
summary(BostonHousing2)

In [ ]:
ggpairs(BostonHousing2[,6:19])

In [ ]:
myboston = BostonHousing2 %>% 
  mutate(town = as.factor(town),
   rad = as.factor(rad),
   chas = as.factor(chas))

In [ ]:
names(myboston)

In [ ]:
str(myboston)

3 možnosti jak ve funkci volat nazev proměnné v data framu:

In [ ]:
# Plot without function:
var1 = "lon"
var2 = "lat"
var3 = "cmedv"

ggplot(BostonHousing2, aes(x=get(var1), y=get(var2))) + 
  geom_point(aes(size=get(var3)))

## 1. Použít textový název a převést ho na název proměnné pomocí get()

In [ ]:
plot_my_scatter = function(df,var1,var2,var3 = "cmedv",var4 = "rad",var5="chas",plot_lm = F){
  my_plot = ggplot(df, aes(x=get(var1), y=get(var2),shape=get(var5), size=get(var3), color=get(var4))) + 
    geom_point() +
    labs(x = var1, y = var2, size = var3, colour = var4, shape = var5)
  if (plot_lm) {my_plot = my_plot + geom_smooth(method=lm, se=F)}
  my_plot
  }

In [ ]:
var1 = "lon"
var2 = "lat"
var3 = "cmedv"
var4 = "rad"
var5 = "chas"

plot_my_scatter(myboston,var1,var2,var3,plot_lm = T)

In [ ]:
var1 = "rm"
var2 = "cmedv"
var3 = "crim"
var4 = "chas"
var5 = "chas"

plot_my_scatter(myboston,var1,var2,var3,var4,var5,plot_lm = T)

In [ ]:
var1 = "nox"
var2 = "cmedv"
var3 = "crim"
var4 = "rad"
var5 = "chas"

plot_my_scatter(myboston,var1,var2,var3,plot_lm = F)

## 2. Použít názvy proměnných a vložit je do {{}}.

In [ ]:
plot_my_scatter2 = function(df,var1,var2,var3,var4,var5){
  ggplot(df, aes(x={{var1}}, y={{var2}},size={{var3}},shape={{var5}}, color={{var4}})) + 
    geom_point()
  }

In [ ]:
plot_my_scatter2(myboston,lon,lat,cmedv,rad,chas)

## 3. Použít názvy proměnných a zavolat j pomocí !!

In [ ]:
plot_my_scatter3 = function(df,var1,var2,var3,var4,var5){
  ggplot(df, aes(x=!!var1, y=!!var2,size=!!var3,shape=!!var5, color=!!var4)) + 
    geom_point()
  }

In [ ]:
var1 = sym("nox")
var2 = sym("cmedv")
var3 = sym("crim")
var4 = sym("rad")
var5 = sym("chas")

plot_my_scatter3(myboston,var1,var2,var3,var4,var5)

Hint to plot map: https://rpubs.com/Mturgal/164857

In [ ]:
install.packages("ggmap")
install.packages("mapproj")

library(ggmap)
library(mapproj)

In [ ]:
BostonLL <-c(-71.30, 42.00, -70.80, 42.40)
map <- get_map(location = BostonLL, zoom = 11)

In [ ]:
mapPoints <- ggmap(map) +
   geom_point(aes(x = lon, y = lat,color=crim),data = myboston, alpha = 0.7,size=3)+
   scale_color_gradient(low="#9ebcda",high="#8856a7")+
   labs(title="Median value of owner-occupied homes in $1000's",y="Latitude",x="Longtitude",color="House Prices (in 1000$'s)" )

mapPoints

In [ ]:
mapPoints <- ggmap(map) +
   geom_point(data = myboston, aes(x=get(var1), y=get(var2),shape=get(var5), color=get(var4),size=get(var3)))+
   labs(colour = var4, shape = var5, size = var3)
mapPoints 

In [ ]:
class(plot_my_scatter(myboston,var1,var2,var3))

In [ ]:
class(   geom_point(aes(x = lon, y = lat,color=crim),data = myboston, alpha = 0.7,size=3))